In [ ]:
import numpy as np
from scipy.stats import gaussian_kde
import plotly.graph_objects as go

# ============================================================
# 1. 테스트용 3D 가우시안 데이터 생성 (두 집단)
# ============================================================
np.random.seed(42)

# 집단 1 설정
mean1 = [0, 0, 0]
cov1 = [
    [1.0, 0.3, 0.2],
    [0.3, 1.2, 0.4],
    [0.2, 0.4, 1.0]
]
samples1 = np.random.multivariate_normal(mean1, cov1, size=1500)
x1, y1, z1 = samples1[:, 0], samples1[:, 1], samples1[:, 2]

# 집단 2 설정 (평균 이동)
mean2 = [1.5, 1.5, 1.5]
cov2 = cov1
samples2 = np.random.multivariate_normal(mean2, cov2, size=1500)
x2, y2, z2 = samples2[:, 0], samples2[:, 1], samples2[:, 2]

# 전체 데이터셋 결합하여 공통 범위 계산
all_x = np.concatenate([x1, x2])
all_y = np.concatenate([y1, y2])
all_z = np.concatenate([z1, z2])

# ============================================================
# 2. KDE 모델 학습 (두 집단 각각)
# ============================================================
kde1 = gaussian_kde(np.vstack([x1, y1, z1]))
kde2 = gaussian_kde(np.vstack([x2, y2, z2]))

# ============================================================
# 3. 공통 3D GRID 생성
# ============================================================
grid_n = 35  # 너무 크면 느려짐
X, Y, Z = np.mgrid[
    all_x.min():all_x.max():grid_n*1j,
    all_y.min():all_y.max():grid_n*1j,
    all_z.min():all_z.max():grid_n*1j
]

coords = np.vstack([X.ravel(), Y.ravel(), Z.ravel()])

# ============================================================
# 4. grid KDE 계산 (두 집단 각각)
# ============================================================
density1 = kde1(coords).reshape(X.shape)
density2 = kde2(coords).reshape(X.shape)

# ============================================================
# 5. Plotly 시각화 (두 집단의 Isosurface)
# ============================================================

fig = go.Figure()

# Isosurface 설정
low_color_percent = 90
surface_count = 4

# --- 집단 1 시각화 ---
isomin_value1 = np.percentile(density1, low_color_percent)

fig.add_trace(go.Isosurface(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=density1.flatten(),
    isomin=isomin_value1,
    isomax=np.percentile(density1, 99.5),
    surface_count=surface_count,
    opacity=0.6,
    colorscale="Plasma",
    name="집단 1 분포 (평균: (0, 0, 0))",
    showscale=False
))

# --- 집단 2 시각화 ---
isomin_value2 = np.percentile(density2, low_color_percent)

fig.add_trace(go.Isosurface(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=density2.flatten(),
    isomin=isomin_value2,
    isomax=np.percentile(density2, 99.5),
    surface_count=surface_count,
    opacity=0.6,
    colorscale="Viridis",
    name="집단 2 분포 (평균: (1.5, 1.5, 1.5))",
    showscale=False
))


fig.update_layout(
    title="두 3D 가우시안 분포의 아이소표면 KDE 비교",
    scene=dict(
        aspectmode="data",
        xaxis_title='X 축',
        yaxis_title='Y 축',
        zaxis_title='Z 축'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    width=900, height=800
)

fig.show()